# Import Data and Libraries

In [4]:
#Import libraries
import pandas as pd
import numpy as np
import os

In [5]:
#Import data
path = r"C:\Users\carol\Documents\Career Foundry\4 Python Fundamentals\Instacart Basket Analysis"
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_derived.pkl'))

In [6]:
#Check columns
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 19 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int32   
 1   user_id                 int32   
 2   order_number            int8    
 3   orders_day_of_week      int8    
 4   orders_time             int8    
 5   days_since_prior_order  float16 
 6   First Order             bool    
 7   product_id              int32   
 8   add_to_cart_order       int16   
 9   reordered               int8    
 10  product_name            object  
 11  aisle_id                int16   
 12  department_id           int8    
 13  prices                  float32 
 14  _merge                  category
 15  price_range_loc         object  
 16  busiest_day             object  
 17  busiest_days            object  
 18  busiest_period_of_day   object  
dtypes: bool(1), category(1), float16(1), float32(1), int16(2), int32(3), int8(5), obje

In [7]:
#Check sample of data
ords_prods_merge.head(10)

order_id  user_id  order_number  orders_day_of_week  orders_time  \
0   2539329        1             1                   2            8   
1   2398795        1             2                   3            7   
2    473747        1             3                   3           12   
3   2254736        1             4                   4            7   
4    431534        1             5                   4           15   
5   3367565        1             6                   2            7   
6    550135        1             7                   1            9   
7   3108588        1             8                   1           14   
8   2295261        1             9                   1           16   
9   2550362        1            10                   4            8   

   days_since_prior_order  First Order  product_id  add_to_cart_order  \
0                     NaN         True         196                  1   
1                    15.0        False         196                  1   
2                    21.0        False         196                  1   
3                    29.0        False         196                  1   
4                    28.0        False         196                  1   
5                    19.0        False         196                  1   
6                    20.0        False         196                  1   
7                    14.0        False         196                  2   
8                     0.0        False         196                  4   
9                    30.0        False         196                  1   

   reordered product_name  aisle_id  department_id  prices _merge  \
0          0         Soda        77              7     9.0   both   
1          1         Soda        77              7     9.0   both   
2          1         Soda        77              7     9.0   both   
3          1         Soda        77              7     9.0   both   
4          1         Soda        77              7     9.0   both   
5          1         Soda        77              7     9.0   both   
6          1         Soda        77              7     9.0   both   
7          1         Soda        77              7     9.0   both   
8          1         Soda        77              7     9.0   both   
9          1         Soda        77              7     9.0   both   

     price_range_loc     busiest_day    busiest_days busiest_period_of_day  
0  Mid-range product  Regularly busy  Regularly busy        Average orders  
1  Mid-range product  Regularly busy      Least busy        Average orders  
2  Mid-range product  Regularly busy      Least busy        Average orders  
3  Mid-range product      Least busy      Least busy        Average orders  
4  Mid-range product      Least busy      Least busy           Most orders  
5  Mid-range product  Regularly busy  Regularly busy        Average orders  
6  Mid-range product  Regularly busy     Busiest day        Average orders  
7  Mid-range product  Regularly busy     Busiest day           Most orders  
8  Mid-range product  Regularly busy     Busiest day        Average orders  
9  Mid-range product      Least busy      Least busy        Average orders

# Step 2

In [8]:
# Calculate the mean of order_number and group data by department id
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

order_number
                      mean
department_id             
1                15.457838
2                17.277920
3                17.170395
4                17.811403
5                15.215751
6                16.439806
7                17.225802
8                15.340650
9                15.895474
10               20.197148
11               16.170638
12               15.887671
13               16.583536
14               16.773669
15               16.165037
16               17.665606
17               15.694469
18               19.310397
19               17.177343
20               16.473447
21               22.902379

# Step 3

The results from the entire dataframe include all 21 departments, whereas the results from the subset only include values from 8 departments. The order number means from the departments covered by the subset are also all slightly different as compared to the means from the entire dataframe.

# Step 4

In [9]:
#Create max order column based on the user_id and order_number columns
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [10]:
#Create a loyalty flag column
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

# Step 5

In [11]:
#Find basic statistics of product prices for each loyalty category
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'median', 'min', 'max']})

prices                     
                       mean median  min      max
loyalty_flag                                    
Loyal customer    10.386335    7.4  1.0  99999.0
New customer      13.294669    7.4  1.0  99999.0
Regular customer  12.495717    7.4  1.0  99999.0

All 3 customer types have the same minimum, median, and maximum prices of products purchased, but loyal customers have a slightly lower average price of purchased products compared to new and regular customers.

# Step 6

Create a spending flag for each user based on the average price across all their orders


In [12]:
# Step 1 - create an average price column based on the user_id and prices columns
ords_prods_merge['avg_order'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [14]:
# Create spender flag
# Low spenders have an average purchase price below 10 
# High spenders have an average purchase price equal to or greater than 10
ords_prods_merge.loc[ords_prods_merge['avg_order'] < 10, 'spender_type'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_order'] >= 10, 'spender_type'] = 'High spender'

In [16]:
ords_prods_merge['spender_type'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spender_type, dtype: int64

In [17]:
#Check results
ords_prods_merge[['user_id', 'prices', 'avg_order', 'spender_type']].head(15)

user_id  prices  avg_order spender_type
0         1     9.0   6.367797  Low spender
1         1     9.0   6.367797  Low spender
2         1     9.0   6.367797  Low spender
3         1     9.0   6.367797  Low spender
4         1     9.0   6.367797  Low spender
5         1     9.0   6.367797  Low spender
6         1     9.0   6.367797  Low spender
7         1     9.0   6.367797  Low spender
8         1     9.0   6.367797  Low spender
9         1     9.0   6.367797  Low spender
10       15     9.0   3.980556  Low spender
11       15     9.0   3.980556  Low spender
12       15     9.0   3.980556  Low spender
13       15     9.0   3.980556  Low spender
14       15     9.0   3.980556  Low spender

# Step 7

Determine frequent vs non-frequent customers

In [18]:
# Create order frequency column based on the user_id and prices columns
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [19]:
#Create flag based on order frequency
#Non-frequent customer - 20 < median
#Regular customer - 10 < median <= 20
#Frequent customer - median <= 10
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] <= 20) & (ords_prods_merge['order_frequency'] > 10), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [20]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [17]:
#Check final results
ords_prods_merge.head()

order_id  user_id  order_number  orders_day_of_week  orders_time  \
0   2539329        1             1                   2            8   
1   2398795        1             2                   3            7   
2    473747        1             3                   3           12   
3   2254736        1             4                   4            7   
4    431534        1             5                   4           15   

   days_since_prior_order  First Order  product_id  add_to_cart_order  \
0                     NaN         True         196                  1   
1                    15.0        False         196                  1   
2                    21.0        False         196                  1   
3                    29.0        False         196                  1   
4                    28.0        False         196                  1   

   reordered  ...    price_range_loc     busiest_day    busiest_days  \
0          0  ...  Mid-range product  Regularly busy  Regularly busy   
1          1  ...  Mid-range product  Regularly busy      Least busy   
2          1  ...  Mid-range product  Regularly busy      Least busy   
3          1  ...  Mid-range product      Least busy      Least busy   
4          1  ...  Mid-range product      Least busy      Least busy   

   busiest_period_of_day max_order  loyalty_flag avg_order spender_type  \
0         Average orders        10  New customer  6.367797  Low spender   
1         Average orders        10  New customer  6.367797  Low spender   
2         Average orders        10  New customer  6.367797  Low spender   
3         Average orders        10  New customer  6.367797  Low spender   
4            Most orders        10  New customer  6.367797  Low spender   

  order_frequency         frequency_flag  
0            20.5  Non-frequent customer  
1            20.5  Non-frequent customer  
2            20.5  Non-frequent customer  
3            20.5  Non-frequent customer  
4            20.5  Non-frequent customer  

[5 rows x 25 columns]

# Export Data

In [18]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_flagged.pkl'))